In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


whilst seeing the output make sure you are aware that the machine laerning model didnt know what an english word was!!


* The structure of the output = a blocksof text  begin with a speaker name which i inputted in the model, in all capital letters similar to the dataset.

* As seeen below, the model is trained on small and large batches of text (100 characters each) and is still able to generate a longer sequence of text with not- so ye in it hahahaha!

In [5]:
import tensorflow as tf

import numpy as np
import os
import time

In [6]:
path_to_file = tf.keras.utils.get_file('SCI-FI.txt', 'http://www.scifiscripts.com/scripts/5thelement.txt')

In [7]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 162771 characters


In [8]:
# Take a look at the first 250 characters in text
print(text[:250])

				The Fifth Element

				An original script
				by
				Luc Besson

				Revisions by
				Luc Besson
				and
				Robert Mark Kamen

				August 1995 Draft

				Gaumont and Les Films du Dauphin

FADE IN:

1	EXT.  DESERT  NILE RIVER  VA


In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

78 unique characters


### Vectorize the texts,

Before training, you need to convert the strings to a numerical representation. 

The ''tf.keras.layers.StringLookup'' layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [10]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

```
# This is formatted as code
```

It converts from tokens to character IDs:

In [12]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[53, 54, 55, 56, 57, 58, 59], [76, 77, 78]]>

inverting this represeentation to human-readable strings is the main task at hand. For this you can use `tf.keras.layers.

1.   List item
2.   List item

StringLookup(..., invert=True)`.

In [13]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [14]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [15]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [16]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## PREDICTING NEXT PROBABLE WORD

Since RNNs maintain an internal state of neurons like cnns that depends on the previously seen elements, given all the characters computed until this moment, we might or might not use "ensembling leaning" next time.


Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.


In [17]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [18]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

	
	
	
	
T
h
e
 
F
i


In [19]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [20]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\t' b'\t' b'\t' b'\t' b'T' b'h' b'e' b' ' b'F' b'i' b'f' b't' b'h' b' '
 b'E' b'l' b'e' b'm' b'e' b'n' b't' b'\r' b'\n' b'\r' b'\n' b'\t' b'\t'
 b'\t' b'\t' b'A' b'n' b' ' b'o' b'r' b'i' b'g' b'i' b'n' b'a' b'l' b' '
 b's' b'c' b'r' b'i' b'p' b't' b'\r' b'\n' b'\t' b'\t' b'\t' b'\t' b'b'
 b'y' b'\r' b'\n' b'\t' b'\t' b'\t' b'\t' b'L' b'u' b'c' b' ' b'B' b'e'
 b's' b's' b'o' b'n' b'\r' b'\n' b'\r' b'\n' b'\t' b'\t' b'\t' b'\t' b'R'
 b'e' b'v' b'i' b's' b'i' b'o' b'n' b's' b' ' b'b' b'y' b'\r' b'\n' b'\t'
 b'\t' b'\t' b'\t' b'L' b'u' b'c' b' '], shape=(101,), dtype=string)


In [21]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\t\t\t\tThe Fifth Element\r\n\r\n\t\t\t\tAn original script\r\n\t\t\t\tby\r\n\t\t\t\tLuc Besson\r\n\r\n\t\t\t\tRevisions by\r\n\t\t\t\tLuc '
b'Besson\r\n\t\t\t\tand\r\n\t\t\t\tRobert Mark Kamen\r\n\r\n\t\t\t\tAugust 1995 Draft\r\n\r\n\t\t\t\tGaumont and Les Films du Dauph'
b'in\r\n\r\nFADE IN:\r\n\r\n1\tEXT.  DESERT  NILE RIVER  VALLEY - DAY\r\n\r\n\tSomewhere in the Nile at the edge of t'
b'he desert.\r\n\r\n\tCREDITS  ROLL\r\n\r\n\tWRITTEN:\tEGYPT 1913\r\n\r\n\tOMAR and his mule zigzag along the bottom of'
b' sun scorched dunes.\r\n\r\n2\tEXT.  TEMPLE  EXCAVATION - DAY\r\n\r\n\tThe mule and the boy finally reach a cam'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [22]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [23]:
split_input_target(list("hunterxhunters"))

(['h', 'u', 'n', 't', 'e', 'r', 'x', 'h', 'u', 'n', 't', 'e', 'r'],
 ['u', 'n', 't', 'e', 'r', 'x', 'h', 'u', 'n', 't', 'e', 'r', 's'])

In [24]:
dataset = sequences.map(split_input_target)

In [25]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\t\t\t\tThe Fifth Element\r\n\r\n\t\t\t\tAn original script\r\n\t\t\t\tby\r\n\t\t\t\tLuc Besson\r\n\r\n\t\t\t\tRevisions by\r\n\t\t\t\tLuc'
Target: b'\t\t\tThe Fifth Element\r\n\r\n\t\t\t\tAn original script\r\n\t\t\t\tby\r\n\t\t\t\tLuc Besson\r\n\r\n\t\t\t\tRevisions by\r\n\t\t\t\tLuc '


### Create training batches

You used `tf.data` to split the text into manageable sequences. 

In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

###BUILD THE MODEL

This section defines the model as a `keras.Model` subclass (For details see 
This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.


^^^ go through each type of rnn to learn the dense layers of embedding. MOST IMP if feeding data into bernard ansd spitting out relevant to the point output.

In [27]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [28]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [29]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).


## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 79) # (batch_size, sequence_length, vocab_size)


In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  20224     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  80975     
                                                                 
Total params: 4,039,503
Trainable params: 4,039,503
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [33]:
sampled_indices

array([34, 76, 32, 47, 34, 66, 27, 45,  7, 24, 23, 62, 46,  6, 13, 34, 10,
       69, 38, 18, 24, 46, 14, 17, 20, 31, 59,  2, 49, 59, 35, 74, 39,  2,
       48,  1,  8, 18, 16, 49, 68, 62, 55, 58, 72, 29, 21, 63, 48, 26, 24,
        9, 54, 54, 14, 36,  4, 61,  7, 19, 39, 42, 12, 67, 68, 39,  1, 56,
       24, 73, 36, 14, 78,  8, 60, 48, 40, 59, 32,  5, 24, 70, 26, 77, 23,
        6, 15, 44, 31, 17, 60, 28, 15, 30, 46, 39, 62,  8, 67, 47])

Decode these to see the text predicted by this

untrained model:

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ho in reloading.\r\n\r\n\t\t\t\t\t\tEMPEROR\r\n\t\t\t\tI'd say about thirty yards to the left.\r\n\r\n\tKorben hefts the "

Next Char Predictions:
 b'HxFUHnAS\':9jT"/H,qL4:T036Eg\nWgIvM\nV\t(42WpjcftC7kV?:)bb0J i\'5MP.opM\td:uJ0z(hVNgF!:r?y9"1RE3hB1DTMj(oU'


## TRAIN THE MODEL

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

## ATTACH THE OPTIMIZER AND LOSS FUNCTION

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.

In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 79)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.3697495, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [37]:
tf.exp(example_batch_mean_loss).numpy()

79.023834

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [38]:
model.compile(optimizer='adam', loss=loss)

## CONFIGURE CHECKPOINTS
 Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:



In [39]:
checkpoint = tf.train.Checkpoint(model)

# Save a checkpoint to /tmp/training_checkpoints-{save_counter}. Every time
# checkpoint.save is called, the save counter is increased.
save_path = checkpoint.save('/tmp/training_checkpoints')

# Restore the checkpointed values to the `model` object.
checkpoint.restore(save_path)

In [40]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/tmp/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## EXECUTE THE TRAINING

In [41]:
EPOCHS = 35

In [42]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/35
25/25 [==============================] - 30s 1s/step - loss: 4.1154
Epoch 2/35
25/25 [==============================] - 33s 1s/step - loss: 2.9296
Epoch 3/35
25/25 [==============================] - 34s 1s/step - loss: 2.5572
Epoch 4/35
25/25 [==============================] - 35s 1s/step - loss: 2.3322
Epoch 5/35
25/25 [==============================] - 31s 1s/step - loss: 2.2006
Epoch 6/35
25/25 [==============================] - 36s 1s/step - loss: 2.0964
Epoch 7/35
25/25 [==============================] - 38s 2s/step - loss: 2.0043
Epoch 8/35
25/25 [==============================] - 35s 1s/step - loss: 1.9201
Epoch 9/35
25/25 [==============================] - 34s 1s/step - loss: 1.8439
Epoch 10/35
25/25 [==============================] - 34s 1s/step - loss: 1.7738
Epoch 11/35
25/25 [==============================] - 35s 1s/step - loss: 1.7130
Epoch 12/35
25/25 [==============================] - 36s 1s/step - loss: 1.6507
Epoch 13/35
25/25 [==============================

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


In [43]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [44]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some 

---

text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a OG writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROHIT:'])
result = [next_char]

for n in range(5000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROHIT:
				..Cimating arrous!  The Pleese for two
				and Destroying but.

						PRESIDENT
				RIGlook aftaid ither the hall.

						MOTHER (V.O.)
				Hill the proble, you doing the lase, Mr. President, peeden
				Mamious is non a moss man. Sell me minute! Megat time!

	The ZFX ant hands to a plame light for Korben attEV

212BINT.  KORBEN'S  APARTMENT - DAY

	The Professor is lowned up.  There's onver control.  Leeloo points lead of crombaro an, oberingats like me.

						KORBEN
					(uppassing here, paws how to flyseed, Ge're all eath
				and color. One all weashed that. Con AND wanting
				importance. The President Juth for the Taltion
				int Korben Dallas Will acrops the Stones...
				Soun our day gottang never tike?

					(turn light)
				I'm laterally. General?

					GEMANDER
				Fornelius, presistures if I didn't fire of
				this isn't the cam go one it.

	The COP Solice hall call one in my face, but it's moving.

2.1	INT.  COCKEIT  FHLOSTON  PARADISE  LOUNDE  LABORATORYE

	The 

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 



In [46]:
start = time.time()
states = None
next_char = tf.constant(['ROHIT:', 'RAHUL:', 'BERNARD:', 'SOLOMON:', 'DOLORES:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROHIT: FHLOSTON  PARADISE - DAY\r\n\r\n\tLeeloo is armuns.  A few alien the farryore platesull.\r\n\r\n\t\t\t\t\t\tLOC RHOD\r\n\t\t\t\tKorben sonight aseed)\r\n\t\t\t\tYou'll leadn, it war! If IGe's the missile.\r\n\r\n\tGave and his excroles from the flame tower day.\r\n\r\n6\tINT.  TEMPLE  ROOM - DABAN\r\n\r\n\tA going space in close?  Leeloo is awarm. Bolich stands to pun in the chamicated.\r\n\r\n\t\t\t\t\t\tKORBEN\r\n\t\t\t\tAny eveny, De forget it?\r\n\r\n\tLoc Rhod hopsing the TaxC TIL  LONDANG  LONDELORGH\r\n\r\n\tThe right is loaking at 50, one book.  Leeloo looks left on.\r\n\r\n\t\t\t\t\t\tKORBEN\r\n\t\t\t\tLife 1-, I'll be abrow.\r\n\r\n\tHe emperse string them one questiona delicate a case.\r\n\r\n\t\t\t\t\t\tKORBEN\r\n\t\t\t\tTalk a million -of I- make\r\n\t\t\tconst left on your latcher.\r\n\r\n\t\t\t\t\t\tKORBEN\r\n\t\t\t\tSo no open for lack. Fither a wellone)\r\n\t\t\t\t...Wh cave  I was or a class!\r\n\t\t\t\tit's how daybuh?\r\n\r\n\t\t\t\t\t\tPRESIDEN

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [47]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [49]:
states = None
next_char = tf.constant(['bernard:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

bernard:
				Comeround an ears out him...

	He pots the CLieftoon it in his hands.  In the chall-drests, 


Advanced: Customized Training


The above training procedure is simple, but does not give you much control. It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use tf.GradientTape to track the gradients. You can learn more about this approach by reading the eager execution guide.

The basic procedure is:

Execute the model and calculate the loss under a tf.GradientTape.
Calculate the updates and apply them to the model using the optimizer.

In [50]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [51]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
    
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

model.fit(dataset, epochs=5)

Epoch 1/5
25/25 [==============================] - 33s 1s/step - loss: 4.0507
Epoch 2/5
25/25 [==============================] - 31s 1s/step - loss: 2.9196
Epoch 3/5
25/25 [==============================] - 31s 1s/step - loss: 2.5367
Epoch 4/5
25/25 [==============================] - 30s 1s/step - loss: 2.3142
Epoch 5/5
25/25 [==============================] - 34s 1s/step - loss: 2.1870


In [53]:
result

[<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'bernard:'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'\r'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'\n'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'\t'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'\t'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'\t'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'\t'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'C'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'o'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'm'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'e'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'r'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'o'], dtype=object)>,
 <tf.Tensor: